In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import beamsim

# Set seaborn style for beautiful plots
sns.set_style("whitegrid")
sns.set_palette("husl")
plt.rcParams["figure.figsize"] = (12, 8)
plt.rcParams["font.size"] = 11

In [ ]:
def generate_yaml_config(topology="gossip", **overrides):
    """
    Generate a YAML configuration with default values and optional overrides.
    
    Args:
        topology: Network topology ("gossip", "grid", etc.)
        **overrides: Dictionary of parameter overrides
        
    Common override parameters:
        - signature_size: Size of signatures in bytes
        - max_incoming_bandwidth: Bandwidth limit (e.g., "10Mbps" or None for unlimited)
        - mesh_n: Gossip mesh_n parameter
        - non_mesh_n: Gossip non_mesh_n parameter
        - group_validator_count: Number of validators per group
        - group_local_aggregator_count: Number of local aggregators per group
        - group_count: Number of groups
        - global_aggregator_count: Number of global aggregators
        - snark1_threshold: Threshold for SNARK1 generation
        - aggregation_rate_per_sec: Rate of signature aggregation
        - random_seed: Random seed for reproducibility
    """
    
    # Default configuration values
    defaults = {
        'backend': 'ns3-direct',
        'snark1_pull': True,
        'snark1_half_direct': True,
        'topology': topology,
        'shuffle': False,
        'random_seed': 42,
        'group_count': 8,
        'group_validator_count': 1024,
        'group_local_aggregator_count': 102,
        'global_aggregator_count': 102,
        'mesh_n': 6,
        'non_mesh_n': 4,
        'signature_time': '20ms',
        'signature_size': 3072,
        'snark_size': 131072,
        'snark1_threshold': 0.75,
        'snark2_threshold': 0.66,
        'aggregation_rate_per_sec': 1000,
        'snark_recursion_aggregation_rate_per_sec': 50,
        'pq_signature_verification_time': '30us',
        'snark_proof_verification_time': '5ms',
        'gml': 'shadow-atlas.bin',
        'max_incoming_bandwidth': '100Mbps'  # Default bandwidth limit
    }
    
    # Apply overrides
    config = {**defaults, **overrides}
    
    # Build the YAML string
    yaml_content = f"""
# Simulation Backend Configuration
backend: {config['backend']}
                                
snark1_pull: {str(config['snark1_pull']).lower()}
snark1_half_direct: {str(config['snark1_half_direct']).lower()}

# Network Topology Configuration
topology: {config['topology']}

# Whether to shuffle validators from the same group to different routers
shuffle: {str(config['shuffle']).lower()}

# Seed for reproducible simulation results
random_seed: {config['random_seed']}

# Role Assignment Configuration
roles:
  group_count: {config['group_count']}
  group_validator_count: {config['group_validator_count']}
  group_local_aggregator_count: {config['group_local_aggregator_count']}
  global_aggregator_count: {config['global_aggregator_count']}

# Gossipsub Network Configuration
gossip:
  mesh_n: {config['mesh_n']}
  non_mesh_n: {config['non_mesh_n']}

# Cryptographic Constants
consts:
  signature_time: {config['signature_time']}
  signature_size: {config['signature_size']}
  snark_size: {config['snark_size']}
  snark1_threshold: {config['snark1_threshold']}
  snark2_threshold: {config['snark2_threshold']}
  aggregation_rate_per_sec: {config['aggregation_rate_per_sec']}
  snark_recursion_aggregation_rate_per_sec: {config['snark_recursion_aggregation_rate_per_sec']}
  pq_signature_verification_time: {config['pq_signature_verification_time']}
  snark_proof_verification_time: {config['snark_proof_verification_time']}

# Network Simulation Parameters
network:
  gml: "{config['gml']}"
"""
    
    # Add bandwidth constraint if specified
    if config['max_incoming_bandwidth'] is not None:
        yaml_content += f"  max_incoming_bandwidth: {config['max_incoming_bandwidth']}\n"
    
    return beamsim.yaml(yaml_content.strip())


def run_parameter_analysis(parameter_name, parameter_values, topologies=["gossip", "grid"], 
                          display_name=None, display_unit="", **base_config):
    """
    Generic function to analyze the effect of changing a parameter on SNARK1 aggregation time.
    
    Args:
        parameter_name: Name of the parameter to vary (e.g., 'signature_size', 'mesh_n')
        parameter_values: List of values to test for the parameter
        topologies: List of network topologies to test
        display_name: Human-readable name for the parameter (defaults to parameter_name)
        display_unit: Unit to display after the parameter value (e.g., "KB", "Mbps")
        **base_config: Base configuration parameters and run kwargs
        
    Returns:
        DataFrame with results
    """
    
    if display_name is None:
        display_name = parameter_name.replace('_', ' ').title()
    
    # Separate run kwargs from config overrides
    run_kwargs_keys = {'mpi', 'c', 'b', 'g', 'gv', 'la', 'ga', 'shuffle', 't', 'local_aggregation_only'}
    run_kwargs = {k: v for k, v in base_config.items() if k in run_kwargs_keys}
    config_overrides = {k: v for k, v in base_config.items() if k not in run_kwargs_keys}
    
    results = []
    
    print(f"Testing {display_name}: {parameter_values}")
    print(f"Topologies: {topologies}")
    print("Running simulations...\n")
    
    for topology in topologies:
        print(f"Testing topology: {topology}")
        
        for param_value in parameter_values:
            param_display = f"{param_value} {display_unit}".strip() if param_value is not None else "Default/Unlimited"
            print(f"  {display_name}: {param_display}")
            
            # Create config with this parameter value
            current_config = {**config_overrides, parameter_name: param_value}
            modified_yaml = generate_yaml_config(topology=topology, **current_config)
            
            # Run simulation with modified config
            current_run_kwargs = {**run_kwargs, 'c': modified_yaml}
            items = beamsim.run(**current_run_kwargs, t=topology, local_aggregation_only=True)
            
            # Extract SNARK1 timing information
            snark1_sent = beamsim.get_snark1_sent(items)
            if snark1_sent and len(snark1_sent[0]) > 0:
                snark1_completion_time = snark1_sent[0][-1] if snark1_sent[0] else 0
                
                # Get simulation info
                _, _, validator_count, snark1_threshold, _ = beamsim.filter_report(items, "info")[0]
                
                results.append({
                    'topology': topology,
                    'topology_display': beamsim.topology_name[topology],
                    'parameter_name': parameter_name,
                    'parameter_value': param_value,
                    'parameter_display': param_display,
                    'snark1_completion_time': snark1_completion_time,
                    'validator_count': validator_count,
                    'snark1_threshold': snark1_threshold,
                })
    
    return pd.DataFrame(results)


def plot_parameter_analysis(results_df, parameter_name, display_name=None, display_unit="", 
                          title_suffix="", figure_size=(14, 8)):
    """
    Plot the results of a parameter analysis.
    
    Args:
        results_df: DataFrame from run_parameter_analysis
        parameter_name: Name of the parameter that was varied
        display_name: Human-readable name for the parameter
        display_unit: Unit for the parameter
        title_suffix: Additional text for the plot title
        figure_size: Size of the plot figure
    """
    
    if display_name is None:
        display_name = parameter_name.replace('_', ' ').title()
    
    # Create the plot
    plt.figure(figsize=figure_size)
    
    # Plot lines for each topology
    topologies = results_df['topology'].unique()
    colors = sns.color_palette("husl", len(topologies))
    
    for i, topology in enumerate(topologies):
        topo_data = results_df[results_df['topology'] == topology].copy()
        
        # Handle None values and sort
        if parameter_name == 'max_incoming_bandwidth':
            # Special handling for bandwidth (None should be at the end)
            non_none_data = topo_data[topo_data['parameter_value'].notna()].copy()
            non_none_data = non_none_data.sort_values('parameter_value')
            none_data = topo_data[topo_data['parameter_value'].isna()]
            
            x_values = list(range(len(non_none_data)))
            y_values = non_none_data['snark1_completion_time'].tolist()
            
            if len(none_data) > 0:
                x_values.append(len(non_none_data))
                y_values.append(none_data.iloc[0]['snark1_completion_time'])
        else:
            # Normal numeric sorting
            topo_data = topo_data.sort_values('parameter_value')
            x_values = list(range(len(topo_data)))
            y_values = topo_data['snark1_completion_time'].tolist()
        
        plt.plot(
            x_values,
            y_values,
            marker='o',
            linewidth=3,
            markersize=8,
            color=colors[i],
            label=f'{topo_data.iloc[0]["topology_display"]} Topology',
            alpha=0.8
        )
    
    # Set axis labels and title
    plt.xlabel(f'{display_name} {display_unit}'.strip(), fontweight='bold', fontsize=12)
    plt.ylabel('SNARK1 Aggregation Time (ms)', fontweight='bold', fontsize=12)
    
    title = f'Effect of {display_name} on SNARK1 Aggregation Time'
    if title_suffix:
        title += f'\n{title_suffix}'
    title += '\n(Local Aggregation Only Mode)'
    plt.title(title, fontweight='bold', fontsize=14)
    
    # Set x-axis tick labels
    if not results_df.empty:
        unique_values = results_df['parameter_display'].unique()
        plt.xticks(range(len(unique_values)), unique_values, rotation=45)
    
    plt.legend(frameon=True, fancybox=True, shadow=True, fontsize=11)
    plt.grid(True, alpha=0.3)
    
    # Add performance annotations
    if len(results_df) > 0:
        min_time = results_df['snark1_completion_time'].min()
        max_time = results_df['snark1_completion_time'].max()
        time_diff = max_time - min_time
        improvement = (time_diff / max_time) * 100
        
        plt.text(0.02, 0.98, 
                f'Max time difference: {time_diff:.1f}ms\n'
                f'Performance improvement: {improvement:.1f}%',
                transform=plt.gca().transAxes,
                verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.8),
                fontsize=10)
    
    plt.tight_layout()
    plt.show()
    
    # Print summary statistics
    print("\n" + "="*60)
    print(f"{display_name.upper()} EFFECT SUMMARY")
    print("="*60)
    
    for topology in topologies:
        topo_data = results_df[results_df['topology'] == topology]
        if len(topo_data) > 0:
            print(f"\n{topo_data.iloc[0]['topology_display']} Topology:")
            print(f"  SNARK1 time range: {topo_data['snark1_completion_time'].min():.1f} - {topo_data['snark1_completion_time'].max():.1f} ms")
            print(f"  Time difference: {topo_data['snark1_completion_time'].max() - topo_data['snark1_completion_time'].min():.1f} ms")
            
            # Show performance for each parameter value
            for _, row in topo_data.iterrows():
                print(f"    {row['parameter_display']:>12}: {row['snark1_completion_time']:.1f} ms")


def analyze_parameter_effect(parameter_name, parameter_values, topologies=["gossip", "grid"], 
                           display_name=None, display_unit="", title_suffix="", **base_config):
    """
    Complete analysis workflow: run simulations and plot results.
    
    Args:
        parameter_name: Name of the parameter to vary
        parameter_values: List of values to test
        topologies: List of topologies to test
        display_name: Human-readable parameter name
        display_unit: Unit for the parameter
        title_suffix: Additional text for plot title
        **base_config: Base configuration and run parameters
        
    Returns:
        DataFrame with results
    """
    
    # Run the analysis
    results_df = run_parameter_analysis(
        parameter_name, parameter_values, topologies, 
        display_name, display_unit, **base_config
    )
    
    # Plot the results
    plot_parameter_analysis(
        results_df, parameter_name, display_name, 
        display_unit, title_suffix
    )
    
    return results_df

In [ ]:
def analyze_signature_size_effect(signature_sizes, topologies=["gossip", "grid"], **base_config):
    """
    Analyze how signature size affects SNARK1 aggregation time using the refactored framework.
    
    Args:
        signature_sizes: List of signature sizes to test (in bytes)
        topologies: List of network topologies to test
        **base_config: Base configuration parameters
        
    Returns:
        DataFrame with results
    """
    return analyze_parameter_effect(
        parameter_name='signature_size',
        parameter_values=signature_sizes,
        topologies=topologies,
        display_name='Signature Size',
        display_unit='bytes',
        title_suffix='',
        **base_config
    )

In [ ]:
def analyze_bandwidth_effect(bandwidth_limits, signature_size=3072, topologies=["gossip", "grid"], **base_config):
    """
    Analyze how max incoming bandwidth affects SNARK1 aggregation time using the refactored framework.
    
    Args:
        bandwidth_limits: List of bandwidth limits to test (in Mbps, None for unlimited)
        signature_size: Fixed signature size to use (in bytes)
        topologies: List of network topologies to test
        **base_config: Base configuration parameters
        
    Returns:
        DataFrame with results
    """
    # Convert bandwidth values to the format expected by the YAML config
    bandwidth_values = []
    for bw in bandwidth_limits:
        if bw is None:
            bandwidth_values.append(None)
        else:
            bandwidth_values.append(f"{bw}Mbps")
    
    return analyze_parameter_effect(
        parameter_name='max_incoming_bandwidth',
        parameter_values=bandwidth_values,
        topologies=topologies,
        display_name='Max Incoming Bandwidth',
        display_unit='',
        title_suffix=f'Signature Size: {signature_size/1024:.1f} KB',
        signature_size=signature_size,  # Fixed signature size
        **base_config
    )

In [ ]:
def analyze_mesh_n_effect(mesh_n_values, topologies=["gossip"], **base_config):
    """
    Analyze how mesh_n (gossip mesh size) affects SNARK1 aggregation time.
    
    Args:
        mesh_n_values: List of mesh_n values to test
        topologies: List of network topologies to test (typically just "gossip")
        **base_config: Base configuration parameters
        
    Returns:
        DataFrame with results
    """
    return analyze_parameter_effect(
        parameter_name='mesh_n',
        parameter_values=mesh_n_values,
        topologies=topologies,
        display_name='Mesh N',
        display_unit='peers',
        title_suffix='Gossip Network Configuration',
        **base_config
    )


def analyze_non_mesh_n_effect(non_mesh_n_values, topologies=["gossip"], **base_config):
    """
    Analyze how non_mesh_n (gossip non-mesh connections) affects SNARK1 aggregation time.
    
    Args:
        non_mesh_n_values: List of non_mesh_n values to test
        topologies: List of network topologies to test (typically just "gossip")
        **base_config: Base configuration parameters
        
    Returns:
        DataFrame with results
    """
    return analyze_parameter_effect(
        parameter_name='non_mesh_n',
        parameter_values=non_mesh_n_values,
        topologies=topologies,
        display_name='Non-Mesh N',
        display_unit='peers',
        title_suffix='Gossip Network Configuration',
        **base_config
    )


def analyze_group_validator_count_effect(validator_counts, topologies=["gossip", "grid"], **base_config):
    """
    Analyze how group_validator_count affects SNARK1 aggregation time.
    
    Args:
        validator_counts: List of validator counts per group to test
        topologies: List of network topologies to test
        **base_config: Base configuration parameters
        
    Returns:
        DataFrame with results
    """
    return analyze_parameter_effect(
        parameter_name='group_validator_count',
        parameter_values=validator_counts,
        topologies=topologies,
        display_name='Group Validator Count',
        display_unit='validators',
        title_suffix='Network Scale Analysis',
        **base_config
    )


def analyze_local_aggregator_count_effect(aggregator_counts, topologies=["gossip", "grid"], **base_config):
    """
    Analyze how group_local_aggregator_count affects SNARK1 aggregation time.
    
    Args:
        aggregator_counts: List of local aggregator counts per group to test
        topologies: List of network topologies to test
        **base_config: Base configuration parameters
        
    Returns:
        DataFrame with results
    """
    return analyze_parameter_effect(
        parameter_name='group_local_aggregator_count',
        parameter_values=aggregator_counts,
        topologies=topologies,
        display_name='Local Aggregator Count',
        display_unit='aggregators',
        title_suffix='Aggregation Capacity Analysis',
        **base_config
    )


def analyze_aggregation_rate_effect(aggregation_rates, topologies=["gossip", "grid"], **base_config):
    """
    Analyze how aggregation_rate_per_sec affects SNARK1 aggregation time.
    
    Args:
        aggregation_rates: List of aggregation rates to test (signatures per second)
        topologies: List of network topologies to test
        **base_config: Base configuration parameters
        
    Returns:
        DataFrame with results
    """
    return analyze_parameter_effect(
        parameter_name='aggregation_rate_per_sec',
        parameter_values=aggregation_rates,
        topologies=topologies,
        display_name='Aggregation Rate',
        display_unit='sig/sec',
        title_suffix='Processing Speed Analysis',
        **base_config
    )

In [ ]:
topologies = ["gossip", "grid"]

# Signature Size Analysis - Using the refactored approach
# This demonstrates how the new framework simplifies the analysis process

# Test different signature sizes to analyze their effect on SNARK1 aggregation time
signature_sizes = [
    1024,    # 1 KB - small signature
    2048,    # 2 KB - medium-small signature  
    3072,    # 3 KB - default signature size
    4096,    # 4 KB - medium signature
    6144,    # 6 KB - large signature
    8192,    # 8 KB - very large signature
]

# Base configuration parameters
base_config = {
    'mpi': False,  # Use MPI for faster simulations
    # Add any other base parameters you want to override
}

# Test on both gossip and grid topologies
topologies = ["gossip", "grid"]

print("Analyzing the effect of signature size on SNARK1 aggregation time...")
print("Using the refactored analysis framework for cleaner code.\n")

# Run the signature size effect analysis using the new framework
results_df = analyze_signature_size_effect(signature_sizes, topologies, **base_config)

In [ ]:
# Analyze the effect of max incoming bandwidth on SNARK1 aggregation time
# Test different bandwidth limits to understand network bottlenecks

bandwidth_limits = [
    1,      # 1 Mbps - Very constrained
    5,      # 5 Mbps - Low bandwidth
    10,     # 10 Mbps - Medium-low bandwidth
    50,     # 50 Mbps - Medium bandwidth
    100,    # 100 Mbps - High bandwidth
    500,    # 500 Mbps - Very high bandwidth
    None    # Unlimited bandwidth (no constraint)
]

# Use fixed signature size of 3072 bytes (3KB) as requested
signature_size = 3072

# Base configuration parameters
base_config = {
    'mpi': False,  # Use MPI for faster simulations
}

# Test on both gossip and grid topologies
topologies = ["gossip", "grid"]

print("Analyzing the effect of max incoming bandwidth on SNARK1 aggregation time...")
print(f"Using fixed signature size: {signature_size} bytes ({signature_size/1024:.1f} KB)")
print("Using the refactored analysis framework.\n")

# Run the bandwidth effect analysis using the new framework
bandwidth_results_df = analyze_bandwidth_effect(bandwidth_limits, signature_size, topologies, **base_config)

In [ ]:
# Example analyses for new parameters using the refactored framework
# These demonstrate how easy it is to analyze different parameters

# Example 1: Analyze mesh_n effect (gossip network mesh size)
print("="*60)
print("EXAMPLE 1: Mesh N Analysis")
print("="*60)

mesh_n_values = [3, 6, 9, 12, 15]  # Different mesh sizes
base_config = {'mpi': False, 'signature_size': 3072}

mesh_n_results = analyze_mesh_n_effect(mesh_n_values, topologies=["gossip"], **base_config)


# Example 2: Analyze non_mesh_n effect (gossip non-mesh connections)
print("\n" + "="*60)
print("EXAMPLE 2: Non-Mesh N Analysis")
print("="*60)

non_mesh_n_values = [2, 4, 6, 8, 10]  # Different non-mesh connection counts
base_config = {'mpi': False, 'signature_size': 3072}

non_mesh_n_results = analyze_non_mesh_n_effect(non_mesh_n_values, topologies=["gossip"], **base_config)


# Example 3: Analyze group_validator_count effect (network scale)
# print("\n" + "="*60)
# print("EXAMPLE 3: Group Validator Count Analysis")
# print("="*60)

# validator_counts = [50, 100, 150, 200, 250]  # Different validator counts per group
# base_config = {'mpi': False, 'signature_size': 3072}

# validator_count_results = analyze_group_validator_count_effect(validator_counts, topologies=["gossip", "grid"], **base_config)


# Example 4: Analyze local aggregator count effect
print("\n" + "="*60)
print("EXAMPLE 4: Local Aggregator Count Analysis")
print("="*60)

aggregator_counts = [1, 10, 100, 256, 512]  # Different local aggregator counts
base_config = {'mpi': False, 'signature_size': 3072}

aggregator_count_results = analyze_local_aggregator_count_effect(aggregator_counts, topologies=["gossip", "grid"], **base_config)

## Refactored Analysis Framework Usage Guide

The notebook has been refactored to eliminate repetitive YAML configuration code. Here's how to use the new framework:

### Core Functions

#### 1. `generate_yaml_config(topology, **overrides)`
- **Purpose**: Generate YAML configuration with defaults and overrides
- **Usage**: `yaml_config = generate_yaml_config("gossip", signature_size=4096, mesh_n=8)`
- **Parameters**: Any simulation parameter can be overridden

#### 2. `analyze_parameter_effect(parameter_name, parameter_values, ...)`
- **Purpose**: Generic function to analyze any parameter's effect
- **Usage**: `results = analyze_parameter_effect('signature_size', [1024, 2048, 4096], mpi=4)`
- **Returns**: DataFrame with results and generates plots automatically

### Pre-built Analysis Functions

#### Network Parameters
- `analyze_signature_size_effect(signature_sizes, ...)` - Signature size analysis
- `analyze_bandwidth_effect(bandwidth_limits, ...)` - Bandwidth analysis

#### Gossip Network Parameters  
- `analyze_mesh_n_effect(mesh_n_values, ...)` - Mesh size analysis
- `analyze_non_mesh_n_effect(non_mesh_n_values, ...)` - Non-mesh connections analysis

#### Scale Parameters
- `analyze_group_validator_count_effect(validator_counts, ...)` - Validator count analysis
- `analyze_local_aggregator_count_effect(aggregator_counts, ...)` - Aggregator count analysis

#### Performance Parameters
- `analyze_aggregation_rate_effect(aggregation_rates, ...)` - Aggregation rate analysis

### Quick Analysis Examples

```python
# Analyze signature size effect
results = analyze_signature_size_effect([1024, 2048, 4096], mpi=4)

# Analyze mesh size effect (gossip only)
results = analyze_mesh_n_effect([3, 6, 9, 12], topologies=["gossip"], mpi=4)

# Analyze validator count effect
results = analyze_group_validator_count_effect([50, 100, 150], mpi=4)

# Custom parameter analysis
results = analyze_parameter_effect(
    'snark1_threshold', [0.5, 0.66, 0.75, 0.8], 
    display_name='SNARK1 Threshold', display_unit='fraction',
    mpi=4
)
```

### Key Benefits

1. **No YAML Repetition**: Configuration is generated automatically
2. **Consistent Plotting**: All analyses use the same plotting style
3. **Easy Extension**: Add new parameters with minimal code
4. **Backward Compatibility**: Old function names still work
5. **Flexible Configuration**: Override any parameter easily

### Available Override Parameters

- `signature_size` - Signature size in bytes
- `max_incoming_bandwidth` - Bandwidth limit (e.g., "10Mbps" or None)
- `mesh_n` - Gossip mesh size
- `non_mesh_n` - Gossip non-mesh connections
- `group_validator_count` - Validators per group
- `group_local_aggregator_count` - Local aggregators per group
- `group_count` - Number of groups
- `snark1_threshold` - SNARK1 threshold
- `aggregation_rate_per_sec` - Aggregation rate
- `random_seed` - Random seed
- And many more...

In [ ]:
# Simple demonstration of the refactored framework
# This shows how easy it is to analyze different parameters

print("REFACTORED FRAMEWORK DEMONSTRATION")
print("="*50)

# Example 1: Test different mesh_n values (gossip network mesh size)
print("\n1. Testing different mesh_n values:")
mesh_n_values = [3, 6, 9]
print(f"   Values to test: {mesh_n_values}")

# This is all you need - no YAML repetition!
# Uncomment the line below to run the actual analysis
# mesh_results = analyze_mesh_n_effect(mesh_n_values, topologies=["gossip"], mpi=2)

print("   → Analysis function ready: analyze_mesh_n_effect()")

# Example 2: Test different validator counts
print("\n2. Testing different group validator counts:")
validator_counts = [50, 100, 150]
print(f"   Values to test: {validator_counts}")

# This is all you need!
# Uncomment the line below to run the actual analysis
# validator_results = analyze_group_validator_count_effect(validator_counts, mpi=2)

print("   → Analysis function ready: analyze_group_validator_count_effect()")

# Example 3: Custom parameter analysis
print("\n3. Custom parameter analysis:")
print("   Parameter: snark1_threshold")
print("   Values: [0.5, 0.66, 0.75, 0.8]")

# Custom analysis with the generic function
# Uncomment the lines below to run the actual analysis
# threshold_results = analyze_parameter_effect(
#     'snark1_threshold', [0.5, 0.66, 0.75, 0.8],
#     display_name='SNARK1 Threshold', display_unit='fraction',
#     mpi=2
# )

print("   → Analysis function ready: analyze_parameter_effect()")

print("\n" + "="*50)
print("BENEFITS OF THE REFACTORED APPROACH:")
print("✓ No repetitive YAML configuration code")
print("✓ Consistent plotting and analysis")
print("✓ Easy to add new parameter analyses")
print("✓ Flexible configuration overrides")
print("✓ Backward compatible with old functions")
print("="*50)